In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from pandas import DataFrame,Series
### Text processing
import nltk
nltk.download('stopwords')
import string
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\laks\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df= pd.read_csv("prods.csv")
df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
product_id       1000 non-null int64
product_name     1000 non-null object
aisle_id         1000 non-null int64
department_id    1000 non-null int64
dtypes: int64(3), object(1)
memory usage: 31.3+ KB


In [5]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Remove all \r\n
    4. Remove all numbers
    5. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    no_punc = [char for char in mess if char not in string.punctuation + string.digits]

    # Join the characters again to form the string.
    no_punc = ''.join(no_punc)
    
    no_punc = no_punc.replace('\r\\n', '')
    
    # Now just remove any stopwords
    
    return (' '.join([word for word in no_punc.split() if word.lower() not in stopwords.words('english')]))   
     


In [6]:
df['product_name'] = df['product_name'].astype('str')
df['product_id'] = df['product_id'].astype('str')
df['department_id'] = df['department_id'].astype('str')

In [7]:
df['product_name'] = df['product_name'].apply(text_process)

In [8]:
df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,AllSeasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Vod...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [9]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Text=df['product_name'].tolist()
tfidf=text.TfidfVectorizer(input=Text,stop_words="english")
matrix=tfidf.fit_transform(Text)
#print(matrix.shape)

In [10]:
sim_unigram=cosine_similarity(matrix)

In [11]:
def get_similar_articles(x, col):
    return ",".join(df[col].loc[x.argsort()[-5:-1]])
df['suggested_products']=[get_similar_articles(x, "product_name") for x in sim_unigram]


In [12]:
def get_similar_articles_ids(x, col):
    return ",".join(df[col].iloc[x.argsort()[-5:-1]])

In [13]:
df['suggested_pro_ids']=[get_similar_articles_ids(x, "product_id") for x in sim_unigram]
df['suggested_dept_ids']=[get_similar_articles_ids(x, "department_id") for x in sim_unigram]

In [14]:
df.head(10)

,product_id,product_name,aisle_id,department_id,suggested_products,suggested_pro_ids,suggested_dept_ids
0,1,Chocolate Sandwich Cookies,61,19,Oreo Cookies Cream Chocolate Frozen Dairy Dess...,"591,576,172,559","1,19,19,19"
1,2,AllSeasons Salt,104,13,"Black Salt Caramel Dark Chocolate Bar,Thin Sta...","921,273,425,463","19,19,6,13"
2,3,Robust Golden Unsweetened Oolong Tea,94,7,Almond Breeze Unsweetened Almond Coconut Milk ...,"738,862,560,569","16,16,7,19"
3,4,Smart Ones Classic Favorites Mini Rigatoni Vod...,38,1,"Classic coke,Ice Cream Cookies Cream,Mini Doub...","649,431,997,774","7,1,1,9"
4,5,Green Chile Anytime Sauce,5,13,"Petite Green Peas,Caramel Sauce,Apple Green Cu...","969,275,846,851","1,19,17,19"
5,6,Dry Nose Oil,11,11,"Dry Pasta Lasagne,Dry Roasted Pistachios,Super...","913,304,379,214","9,19,5,11"
6,7,Pure Coconut Water Orange,98,7,"Pompelmo Water,Coconut Drink Vanilla,Organic A...","491,670,299,920","21,16,7,16"
7,8,Cut Russet Potatoes Steam N Mash,116,1,"Instant Mashed Potatoes,Gold Potatoes,Zita Cut...","278,306,139,799","9,4,9,15"
8,9,Light Strawberry Blueberry Yogurt,120,16,"NonFat Blueberry Bottom Greek Yogurt,Organic B...","264,584,419,132","16,16,16,16"
9,10,Sparkling Orange Juice Prickly Pear Beverage,115,7,"Sparkling Blush Grape Juice,Organic Orange Tur...","167,488,334,618","7,7,7,7"


In [15]:
df[df['product_id'] == '41']

,product_id,product_name,aisle_id,department_id,suggested_products,suggested_pro_ids,suggested_dept_ids
40,41,Organic Sourdough Einkorn Crackers Rosemary,78,19,"Supreme Pretzel Crackers,Foldit Flatbreads Art...","615,660,363,145","19,3,19,19"


In [16]:
df.to_csv("Submission_capstone2.csv", index=False)